In [43]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as io
import pandas as pd
from pandas import DataFrame as dframe

In [44]:
#対象被験者の特徴ベクトルをMATLABから入手
feat_l_tr = np.loadtxt("feat_left_A01T.txt",delimiter=",")
feat_r_tr = np.loadtxt("feat_right_A01T.txt",delimiter=",")
feat_Test_C1 = np.loadtxt("feat_test_left_A01T.txt",delimiter=",")
feat_Test_C2 = np.loadtxt("feat_test_right_A01T.txt",delimiter=",")

In [45]:
feat_l_te = feat_Test_C1;
feat_r_te = feat_Test_C2;
feat_l_te = feat_l_te.T
feat_r_te = feat_r_te.T

In [46]:
def Dic(feat_l_tr,feat_r_tr): #入力の配列は2次元配列である必要がある
    feat_tr = np.concatenate([feat_l_tr, feat_r_tr],0)
    return feat_tr

In [47]:
#特徴ベクトルを2次元に変換
#特徴ベクトル g は２M×Tになっている必要がある
#Tは被験者の試行回数である
feat_tr = Dic(feat_l_tr,feat_r_tr).T

In [48]:
print(feat_l_tr.shape,feat_l_te.shape)

(30, 4) (4, 30)


In [49]:
def SRC(feat_l_tr,fea_r_tr,feat_tr,feat_l_te): #rightの時は(feat_r_tr,feat_tr,feat_r_te)
    import cvxpy as cv
    ε=0.3
    SRC_Cla = np.zeros((feat_tr.shape[1], feat_l_te.shape[1]))
    SRC_l_Cla = np.zeros((feat_l_tr.shape[0], feat_l_te.shape[1]))
    SRC_r_Cla = np.zeros((feat_l_tr.shape[0], feat_l_te.shape[1]))  #(feat_r_te.shape[0], feat_r_te.shape[1])でもいい
    answer = np.zeros(feat_l_te.shape[1])
    for i in range(feat_l_te.shape[1]):
        x = cv.Variable((feat_tr.shape[1],1)) #目的関数の次元
        objective = cv.Minimize(cv.norm(x,1))
        constraints = [feat_l_te[:,i,np.newaxis] == feat_tr@x]
        prob = cv.Problem(objective, constraints)
        result = prob.solve()
        #classVariableから値を取るためには.valueを付ける
        SRC_Cla[:,i] = np.squeeze(x.value)
        #パラメータxを各クラスのパラメータに分ける
        SRC_l_Cla[:,i] = SRC_Cla[0:feat_l_tr.shape[0],i]
        SRC_r_Cla[:,i] = SRC_Cla[feat_l_tr.shape[0]:SRC_Cla.shape[0],i]
        #分類では
        #配列xの内、右のクラスのみに関する配列を使用して作成したy(右)、左のクラスのみに関する配列を使用して作成したy(左)の
        #残差が小さい方のクラスを割り当てる
    #パラメータを使って擬似的に各クラスのテスト信号の特徴ベクトルを作成
    #sparce性(第一小数以上の値を持たないものは０にする)を重要視する場合入れる
    #SRC_l_Cla = np.asarray(SRC_l_Cla*10, dtype = int)
    #SRC_l_Cla = SRC_l_Cla*0.1
    #SRC_r_Cla = np.asarray(SRC_r_Cla*10, dtype = int)
    #SRC_r_Cla = SRC_r_Cla*0.1
    dout_feat_l_te = np.zeros((feat_l_te.shape[0], feat_l_te.shape[1]))
    dout_feat_r_te = np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #(feat_r_te.shape[0], feat_r_te.shape[1])でもいい
    for i in range(feat_l_te.shape[1]):
        dout_feat_l_te[:,i] = feat_l_tr.T@SRC_l_Cla[:,i]
    for i in range(feat_r_te.shape[1]):
        dout_feat_r_te[:,i] = feat_r_tr.T@SRC_r_Cla[:,i]
    #作成した擬似特徴ベクトルとの残差を求める
    R_l =  np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #残差を格納するための配列
    R_r =  np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #(feat_r_te.shape[0], feat_r_te.shape[1])でもいい
    for i in range(feat_l_te.shape[1]):
        R_l[:,i] = feat_l_te[:,i]-dout_feat_l_te[:,i]
        R_r[:,i] = feat_l_te[:,i]-dout_feat_r_te[:,i]
        if np.linalg.norm(R_l[:,i], ord=2) < np.linalg.norm(R_r[:,i], ord=2):
            answer[i] = 1
        elif np.linalg.norm(R_l[:,i], ord=2) > np.linalg.norm(R_r[:,i], ord=2):
            answer[i] = 2
    return answer

In [50]:
answer = SRC(feat_l_tr,feat_r_tr,feat_tr,feat_l_te)
x = 0
for i in range(answer.shape[0]):
    if answer[i] == 1:
        x = x + 1
print((x/answer.shape[0])*100)
print(answer)

6.666666666666667
[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2.]


In [51]:
answer = SRC(feat_l_tr,feat_r_tr,feat_tr,feat_r_te)
x = 0
for i in range(answer.shape[0]):
    if answer[i] == 2:
        x = x + 1
print((x/answer.shape[0])*100)

83.33333333333334
